In [1]:
#!/usr/bin/env python
"""
Very simple HTTP server in python.
Usage::
    ./dummy-web-server.py [<port>]
Send a GET request::
    curl http://localhost
Send a HEAD request::
    curl -I http://localhost
Send a POST request::
    curl -d "foo=bar&bin=baz" http://localhost
"""
from pathlib import Path    
from http.server import BaseHTTPRequestHandler, HTTPServer
import socketserver
import subprocess
import os
import requests 
import json

class S(BaseHTTPRequestHandler):
    def _set_headers(self):
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()

    def do_GET(self):
        self._set_headers()
        self.wfile.write("<html><body><h1>hi!</h1></body></html>")

    def do_HEAD(self):
        self._set_headers()
        
    def do_POST(self):
        # Doesn't do anything with posted data
        flinkEndpoint="138.4.7.94:8082"
        entryClass="org.fiware.cosmos.orion.flink.cep.CEPMonitoring2"
        content_length = int(self.headers['Content-Length']) # <--- Gets the size of data
        post_data = self.rfile.read(content_length) # <--- Gets the data itself
        print(post_data)
        ##Call function to generate cep code then call to execute maven, teh code must be in the 
        #current directory
        self.execute_maven()
        
        jarId=self.upload_job(flinkEndpoint)
        sel.run_job(jarId,flinkEndpoint,entryClass)
        
        self._set_headers()
        self.wfile.write("<html><body><h1>POST!</h1></body></html>")
    
    def execute_maven(self):
        p = subprocess.Popen(["mvn", "compile test package"], stdout=subprocess.PIPE)
        output, err = p.communicate()
        print (output)
    
    def upload_job(self,flinkEndpoint):
        mypath = Path().absolute()
        FLINK_ENDPOINT = "http://"+flinkEndpoint+"/jars/upload"
        file_list = [  
        ('jarfile', ('orion.flink.cep-1.0-SNAPSHOT.jar', open('orion.flink.cep-1.0-SNAPSHOT.jar', 'rb'), mypath))]
        r = requests.post(url = FLINK_ENDPOINT,  files=file_list) 
        pastebin_url = json.loads(r.text) 
        args=pastebin_url.get("filename").split("/")
        jarId=args[len(args)-1]
        print("About Uploaded Jar:%s"%pastebin_url)
        return jarId
   
    def run_job(self,jarId,flinkEndpoint,entryClass):
        if(entryClass==None || entryClass==''):
            entryClass="org.fiware.cosmos.orion.flink.cep.CEPMonitoring2"
        FLINK_ENDPOINT = "http://"+flinkEndpoint+"/jars/"+jarId+"/run?allowNonRestoredState=true"
        body= {
          "type" : "object",
          "id" : "urn:jsonschema:org:apache:flink:runtime:webmonitor:handlers:JarRunRequestBody",
          "properties" : {
            "entryClass" : entryClass
          }
        } 
        data = json.dumps(body)
        r = requests.post(url = FLINK_ENDPOINT,data=data) 
        pastebin_url = json.loads(r.text)
        jobID=pastebin_url.get("id")
        print("About the running Job:%s"%pastebin_url)    
        return jobID
        
        
def run(server_class=HTTPServer, handler_class=S, port=8092):
    server_address = ('', port)
    httpd = server_class(server_address, handler_class)
    print ('Starting httpd...')
    httpd.serve_forever()

if __name__ == "__main__":
    from sys import argv

    if len(argv) == 2:
        run(port=int(argv[1]))
    else:
        run()

Starting httpd...


KeyboardInterrupt: 

In [ ]:
os.getcwd()

In [ ]:
os.path.dirname(os.path.abspath(__file__))


In [4]:
 from pathlib import Path    

In [50]:
import requests 
import json

mypath = Path().absolute()
print('Absolute path : {}'.format(mypath))
        
def upload_job(flinkEndpoint):
    mypath = Path().absolute()
    FLINK_ENDPOINT = "http://"+flinkEndpoint+"/jars/upload"
    file_list = [  
    ('jarfile', ('orion.flink.cep-1.0-SNAPSHOT.jar', open('orion.flink.cep-1.0-SNAPSHOT.jar', 'rb'), mypath))]
    r = requests.post(url = FLINK_ENDPOINT,  files=file_list) 
    pastebin_url = json.loads(r.text) 
    args=pastebin_url.get("filename").split("/")
    jarId=args[len(args)-1]
    print("About Uploaded Jar:%s"%pastebin_url)

upload_job("138.4.7.94:8082")

Absolute path : /home/amunoz/Desktop/fiware-usage-control/ptp/docker
About Uploaded Jar:{'filename': '/tmp/flink-web-2d49b431-eb90-42de-950c-cf8bdd15bb7b/flink-web-upload/e307009b-9a90-42a1-bd22-5476c9e1ddd7_orion.flink.cep-1.0-SNAPSHOT.jar', 'status': 'success'}


In [52]:
def run_job(self,jarId,flinkEndpoint,entryClass=''):
        if(entryClass==None or entryClass==''):
            entryClass="org.fiware.cosmos.orion.flink.cep.CEPMonitoring.class"
        FLINK_ENDPOINT = "http://"+flinkEndpoint+"/jars/"+jarId+"/run?entry-class=org.fiware.cosmos.orion.flink.cep.CEPMonitoring&allowNonRestoredState=true"
        r = requests.post(url = FLINK_ENDPOINT) 
        pastebin_url = json.loads(r.text)
        jobID=pastebin_url.get("id")
        print("About the running Job:%s"%pastebin_url)    
        return jobID

run_job("","e307009b-9a90-42a1-bd22-5476c9e1ddd7_orion.flink.cep-1.0-SNAPSHOT.jar","138.4.7.94:8082")

About the running Job:{'errors': ["org.apache.flink.client.program.ProgramInvocationException: The program's entry point class 'org.fiware.cosmos.orion.flink.cep.CEPMonitoring' was not found in the jar file."]}


In [42]:

print(data)


{"type": "object", "id": "urn:jsonschema:org:apache:flink:runtime:webmonitor:handlers:JarRunRequestBody", "properties": {"entryClass": "org.fiware.cosmos.orion.flink.cep.CEPMonitoring2"}}
